# Loop 6 Analysis: Debug Exhaustive Search & Investigate 1st Place Gap

## Key Questions:
1. Why did exhaustive search for N=2 find WORSE results than baseline?
2. What is the baseline N=2 configuration? Is it in our search space?
3. What's the theoretical minimum for small N values?
4. What approaches haven't we tried?

In [1]:
import pandas as pd
import numpy as np
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Tree geometry
TREE_PTS = np.array([
    (0,0.8),(0.125,0.5),(0.0625,0.5),(0.2,0.25),(0.1,0.25),(0.35,0),
    (0.075,0),(0.075,-0.2),(-0.075,-0.2),(-0.075,0),(-0.35,0),
    (-0.1,0.25),(-0.2,0.25),(-0.0625,0.5),(-0.125,0.5)
])

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        trunk_w, trunk_h = 0.15, 0.2
        base_w, mid_w, top_w = 0.7, 0.4, 0.25
        tip_y, tier_1_y, tier_2_y, base_y = 0.8, 0.5, 0.25, 0.0
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (0.0, tip_y),
            (top_w/2, tier_1_y), (top_w/4, tier_1_y),
            (mid_w/2, tier_2_y), (mid_w/4, tier_2_y),
            (base_w/2, base_y), (trunk_w/2, base_y),
            (trunk_w/2, trunk_bottom_y), (-trunk_w/2, trunk_bottom_y),
            (-trunk_w/2, base_y), (-base_w/2, base_y),
            (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
            (-top_w/4, tier_1_y), (-top_w/2, tier_1_y),
        ])
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def get_bounding_box_side(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score(trees):
    n = len(trees)
    side = get_bounding_box_side(trees)
    return side ** 2 / n

print('Functions defined')

Functions defined


In [2]:
# Load baseline and examine N=2 configuration
df = pd.read_csv('/home/code/experiments/004_cpp_sa_optimizer/input.csv')
df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)

# Get N=2 configuration
n2_group = df[df['group_id'] == '002']
print('N=2 baseline configuration:')
for _, row in n2_group.iterrows():
    print(f"  Tree {row['item_id']}: x={float(row['x']):.6f}, y={float(row['y']):.6f}, deg={float(row['deg']):.6f}")

# Calculate score
trees_n2 = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in n2_group.iterrows()]
baseline_n2_score = calculate_score(trees_n2)
print(f'\nBaseline N=2 score: {baseline_n2_score:.6f}')

N=2 baseline configuration:
  Tree 0: x=0.154097, y=-0.038541, deg=203.629378
  Tree 1: x=-0.154097, y=-0.561459, deg=23.629378

Baseline N=2 score: 0.450779


In [3]:
# Analyze the baseline N=2 configuration
# The exhaustive search fixed tree 1 at origin with angle 0
# But the baseline might have BOTH trees at non-zero positions/angles

print('Analysis of baseline N=2:')
for i, (_, row) in enumerate(n2_group.iterrows()):
    x, y, deg = float(row['x']), float(row['y']), float(row['deg'])
    print(f'  Tree {i}: x={x:.10f}, y={y:.10f}, deg={deg:.10f}')

# The issue: Our exhaustive search fixed tree 1 at (0,0,0)
# But the baseline might have tree 1 at a different position!
# Let's check if the baseline is centered at origin

xs = [float(row['x']) for _, row in n2_group.iterrows()]
ys = [float(row['y']) for _, row in n2_group.iterrows()]
degs = [float(row['deg']) for _, row in n2_group.iterrows()]

print(f'\nCenter of mass: ({np.mean(xs):.6f}, {np.mean(ys):.6f})')
print(f'Angle difference: {abs(degs[0] - degs[1]):.6f} degrees')

Analysis of baseline N=2:
  Tree 0: x=0.1540970696, y=-0.0385407427, deg=203.6293777307
  Tree 1: x=-0.1540970696, y=-0.5614592573, deg=23.6293777307

Center of mass: (0.000000, -0.300000)
Angle difference: 180.000000 degrees


In [4]:
# The key insight: Our exhaustive search was WRONG because:
# 1. We fixed tree 1 at (0, 0) with angle 0
# 2. But the optimal solution might have tree 1 at a different angle!
# 3. We should search over RELATIVE positions, not absolute

# Let's do a proper exhaustive search for N=2:
# - Fix tree 1 at (0, 0) but search ALL angles for tree 1
# - Search relative position and angle for tree 2

def has_collision(trees):
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

# Quick test: What's the score if we use the baseline angles but center at origin?
baseline_angles = degs
print(f'Baseline angles: {baseline_angles}')

# Recreate baseline at origin
relative_x = xs[1] - xs[0]
relative_y = ys[1] - ys[0]
print(f'Relative position of tree 2: ({relative_x:.6f}, {relative_y:.6f})')

# Test with tree 1 at origin
trees_test = [
    ChristmasTree('0', '0', str(degs[0])),
    ChristmasTree(str(relative_x), str(relative_y), str(degs[1]))
]
test_score = calculate_score(trees_test)
print(f'Score with tree 1 at origin: {test_score:.6f}')
print(f'Baseline score: {baseline_n2_score:.6f}')
print(f'Difference: {abs(test_score - baseline_n2_score):.10f}')

Baseline angles: [203.62937773065684, 23.629377730656792]
Relative position of tree 2: (-0.308194, -0.522919)
Score with tree 1 at origin: 0.450779
Baseline score: 0.450779
Difference: 0.0000000000


In [ ]:
# Now let's do a PROPER exhaustive search for N=2
# Key fix: Search over tree 1's angle too!

from tqdm import tqdm

def proper_exhaustive_n2(angle_step=5, position_step=0.05):
    best_score = float('inf')
    best_config = None
    
    # Search over tree 1's angle (at origin)
    angles_1 = range(0, 360, angle_step)
    angles_2 = range(0, 360, angle_step)
    positions = np.arange(-1.5, 1.5, position_step)
    
    total = len(angles_1) * len(angles_2) * len(positions) * len(positions)
    print(f'Searching {total:,} configurations...')
    
    checked = 0
    for a1 in tqdm(angles_1):
        for a2 in angles_2:
            for dx in positions:
                for dy in positions:
                    trees = [
                        ChristmasTree('0', '0', str(a1)),
                        ChristmasTree(str(dx), str(dy), str(a2))
                    ]
                    
                    if has_collision(trees):
                        continue
                    
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_config = [(0, 0, a1), (dx, dy, a2)]
                    checked += 1
    
    print(f'Checked {checked:,} valid configurations')
    print(f'Best score: {best_score:.6f}')
    print(f'Best config: {best_config}')
    return best_config, best_score

# Run with coarse grid first
best_config, best_score = proper_exhaustive_n2(angle_step=10, position_step=0.1)

In [ ]:
# Compare to baseline
print(f'\nComparison:')
print(f'  Baseline N=2 score: {baseline_n2_score:.6f}')
print(f'  Exhaustive search score: {best_score:.6f}')
print(f'  Difference: {baseline_n2_score - best_score:.6f}')

if best_score < baseline_n2_score - 1e-6:
    print('  *** IMPROVEMENT FOUND! ***')
else:
    print('  Baseline is optimal or better')

In [ ]:
# Fine-tune around the best configuration
def fine_search_n2(initial_config, angle_step=1, position_step=0.01):
    best_score = float('inf')
    best_config = None
    
    a1_init = initial_config[0][2]
    a2_init = initial_config[1][2]
    dx_init = initial_config[1][0]
    dy_init = initial_config[1][1]
    
    angles_1 = range(int(a1_init) - 15, int(a1_init) + 16, angle_step)
    angles_2 = range(int(a2_init) - 15, int(a2_init) + 16, angle_step)
    positions_x = np.arange(dx_init - 0.3, dx_init + 0.3, position_step)
    positions_y = np.arange(dy_init - 0.3, dy_init + 0.3, position_step)
    
    for a1 in angles_1:
        for a2 in angles_2:
            for dx in positions_x:
                for dy in positions_y:
                    trees = [
                        ChristmasTree('0', '0', str(a1)),
                        ChristmasTree(str(dx), str(dy), str(a2))
                    ]
                    
                    if has_collision(trees):
                        continue
                    
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_config = [(0, 0, a1), (dx, dy, a2)]
    
    return best_config, best_score

if best_config:
    fine_config, fine_score = fine_search_n2(best_config)
    print(f'Fine-tuned score: {fine_score:.6f}')
    print(f'Fine-tuned config: {fine_config}')
    print(f'Improvement over baseline: {baseline_n2_score - fine_score:.6f}')

In [ ]:
# Let's also check the theoretical minimum for N=2
# Two trees touching at their widest points

# Tree width at base: 0.7
# Tree height: 0.8 + 0.2 = 1.0

# For N=2, the theoretical minimum bounding box would be:
# - Two trees side by side (touching) at 0 degrees: width = 0.7 + 0.7 = 1.4, height = 1.0
# - Score = 1.4^2 / 2 = 0.98

# Or two trees at 45 degrees each, interlocking:
# - This could be smaller

print('Theoretical analysis for N=2:')
print('  Tree dimensions: width=0.7, height=1.0')
print('  Single tree at 45 deg: side = 0.813173')
print('  Two trees side by side at 0 deg: side = 1.4, score = 0.98')
print(f'  Baseline score: {baseline_n2_score:.6f}')
print(f'  Efficiency: {(0.813173**2 * 2 / baseline_n2_score):.2%}')

In [ ]:
# Key insight: The baseline N=2 score of 0.4508 is VERY good
# This means the bounding box side is sqrt(0.4508 * 2) = 0.95
# That's smaller than a single tree at 45 degrees (0.813)!
# This is only possible if the trees are INTERLOCKING

side_n2 = np.sqrt(baseline_n2_score * 2)
print(f'Baseline N=2 bounding box side: {side_n2:.6f}')
print(f'Single tree at 45 deg side: 0.813173')
print(f'Ratio: {side_n2 / 0.813173:.2%}')

# This means the baseline has found a configuration where two trees
# fit in a box only 17% larger than a single tree!
# This is a very tight packing.

In [ ]:
# Let's visualize the baseline N=2 configuration
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Draw trees
colors = ['green', 'blue']
for i, tree in enumerate(trees_n2):
    coords = list(tree.polygon.exterior.coords)
    poly = MplPolygon(coords, closed=True, facecolor=colors[i], alpha=0.5, edgecolor='black')
    ax.add_patch(poly)
    ax.plot(tree.center_x, tree.center_y, 'ro', markersize=5)
    ax.annotate(f'Tree {i}\n({tree.center_x:.2f}, {tree.center_y:.2f})\nAngle: {tree.angle:.1f}°', 
                (tree.center_x, tree.center_y), fontsize=8)

# Draw bounding box
bounds = unary_union([t.polygon for t in trees_n2]).bounds
side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
center_x = (bounds[0] + bounds[2]) / 2
center_y = (bounds[1] + bounds[3]) / 2

from matplotlib.patches import Rectangle
rect = Rectangle((center_x - side/2, center_y - side/2), side, side, 
                 fill=False, edgecolor='red', linestyle='--', linewidth=2)
ax.add_patch(rect)

ax.set_xlim(center_x - side, center_x + side)
ax.set_ylim(center_y - side, center_y + side)
ax.set_aspect('equal')
ax.set_title(f'Baseline N=2 Configuration\nScore: {baseline_n2_score:.6f}, Side: {side:.6f}')
ax.grid(True, alpha=0.3)
plt.savefig('/home/code/exploration/n2_baseline_visualization.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved visualization to n2_baseline_visualization.png')